In [8]:
%defaultDatasource jdbc:h2:mem:db

In [15]:
DROP TABLE IF EXISTS Drug;
DROP TABLE IF EXISTS Pathology;
DROP TABLE IF EXISTS DrugUse;
DROP TABLE IF EXISTS SideEffect;

CREATE TABLE Drug (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/faers-2017/drug.csv');

CREATE TABLE Pathology (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/faers-2017/pathology.csv');

CREATE TABLE DrugUse (
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  codeDrug INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(codeDrug)
    REFERENCES Drug(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    idPerson,
    codePathology,
    codeDrug
FROM CSVREAD('../../../data/faers-2017/drug-use.csv');

CREATE TABLE SideEffect(
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT idPerson, codePathology
FROM CSVREAD('../../../data/faers-2017/sideeffect.csv');

org.h2.jdbc.JdbcSQLException:  Cannot drop "DRUGUSE" because "INTERACAOMEDICAMENTOS" depends on it; SQL statement

In [7]:
SELECT * FROM Drug;
SELECT * FROM Pathology;
SELECT * FROM DrugUse;
SELECT * FROM SideEffect;

# FDA Adverse Event Reporting System (FAERS)

[FDA Adverse Event Reporting System (FAERS)](https://open.fda.gov/data/faers/).

Fonte de dados: https://www.fda.gov/drugs/fda-adverse-event-reporting-system-faers/fda-adverse-event-reporting-system-faers-latest-quarterly-data-files

# Interação entre medicamentos
Considere como uma interação toda vez que um medicamento A é tomado em conjunto com um medicamento B por uma pessoa. Crie uma consulta que retorne para cada par de medicamento A,B o número de interações, somente para aqueles medicamentos que são tomados em conjunto mais de 30 vezes (passaremos a chamar de **grupo mais 30**).

In [11]:
DROP VIEW IF EXISTS InteracaoMedicamentos;
DROP VIEW IF EXISTS InteracaoMedicamentosMaior30;

CREATE VIEW InteracaoMedicamentos AS
SELECT USOA.codeDrug druga, USOB.codeDrug drugb
FROM DrugUse USOA, DrugUse USOB
WHERE USOA.IdPerson = USOB.IdPerson AND USOA.codeDrug <> USOB.codeDrug;



CREATE VIEW InteracaoMedicamentosMaior30 AS
SELECT druga, drugb, count(*) QuantidadeInteracao
FROM InteracaoMedicamentos
GROUP BY druga, drugb
HAVING QuantidadeInteracao > 30;

SELECT * FROM InteracaoMedicamentosMaior30;

## Tabela Resultante

Se estiver demorando muito para gerar a tabela, você poderá usar a seguinte tabela resultante pronta. Essa tabela tem todas as interações, você ainda precisará filtrar o **grupo mais 30**.

In [13]:
CREATE TABLE DrugInteraction (
  druga INT NOT NULL,
  drugb INT NOT NULL,
  weight INT NOT NULL,
  PRIMARY KEY(druga, drugb)
) AS SELECT
    druga,
    drugb,
    weight
FROM CSVREAD('../../../data/faers-2017/results/drug-interaction.csv');

In [14]:
CREATE VIEW DrugInteraction30 AS
SELECT * FROM DrugInteraction DI
WHERE DI.weight > 30;

SELECT * FROM DrugInteraction30;

# Suporte

### Qual o suporte que cada uma das associações tem?

Gere uma consulta que apresente o suporte do **grupo mais 30**. Ordene o resultado por ordem de suporte. 

Observação: Para que o resultado da divisão seja fracionária, converta o numerador em `DOUBLE` usando:
~~~sql
CAST(campo AS DOUBLE)
~~~

In [21]:
SELECT druga,drugb,weight,CAST(weight AS DOUBLE) / 9073 Suporte
       FROM DrugInteraction30 
       ORDER BY Suporte;

# Confiança

### Qual a confiança que cada uma das associações tem?

Gere uma consulta que apresente a confiança do **grupo mais 30**. Ordene o resultado por ordem de confiança.

In [22]:
DROP VIEW IF EXISTS Confianca;
CREATE VIEW Confianca AS 
SELECT DI30.druga, DI30.drugb, DI30.weight, CAST(DI30.weight AS DOUBLE)/CAST(DIT.UsoTotalA AS DOUBLE) Confianca
FROM DrugInteraction30 DI30, DrugInteractionTotal DIT
WHERE DI30.druga = DIT.druga
ORDER BY Confianca DESC;

SELECT * FROM Confianca;

# Efeito Colateral

Para cada medicamento, qual o percentual do medicamento A que tem efeito colateral junto com o medicamento B, quando comparado com seu uso sem o medicamento B.

In [ ]:
DROP VIEW IF EXISTS EfeitoColateralConj;

CREATE VIEW EfeitoColateralConj AS
SELECT DU.codeDrug DrugA, DU2.CodeDrug DrugB, COUNT(*) Quantidade
FROM DrugUse DU, SideEffect SE, DrugUse DU2
WHERE SE.idPerson = DU.idPerson AND S.idPerson = DU2.idPerson AND DU.codeDrug <> DU2.codeDrug
GROUP BY DrugA, DrugB;

SELECT * FROM EfeitoColateralConj;

In [ ]:

CREATE VIEW EfeitoColateralDiferenca AS
SELECT EC1.DrugA, EC1.DrugB, SUM(EC1.Quantidade) diferenca
FROM  EfeitoColateralConj EC1,  EfeitoColateralConj EC2
WHERE EC1.DrugA = EC2.DrugA and EC1.DrugB <> EC2.DrugB
GROUP BY EC1.DrugA, EC1.DrugB;

CREATE VIEW EfeitoColateralProporcao AS 
SELECT EC.drugA, EC.drugB, CAST(EC.many AS DOUBLE)/ECD.diferenca proporcaoColateral
FROM  EfeitoColateralConj EC, EfeitoColateralDiferenca  ECD
WHERE EC.drugA = ECD.drugA and EC.drugB = ECD.drugB;

SELECT * FROM EfeitoColateralProporcao;